In [1]:
import pandas as pd

# Load dataframe
df = pd.read_csv("student-por.csv", sep=';')

print(df.shape)

df.head()

(649, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [2]:
from sklearn.model_selection import train_test_split

X = df[['failures', 'absences', 'G3']]
y = df['studytime']

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [3]:
X.head()

,failures,absences,G3
0,0,4,11
1,0,2,11
2,0,6,12
3,0,0,14
4,0,0,13


In [4]:
X['G3'] = X['G3'].div(2)
X.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,failures,absences,G3
0,0,4,5.5
1,0,2,5.5
2,0,6,6.0
3,0,0,7.0
4,0,0,6.5


In [5]:
# Find best fitting RandomForestRegressor model 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
import warnings
warnings.filterwarnings('ignore')

param_grid = {
    'n_estimators': np.arange(50, 150, 10), 
    'max_features': np.arange(1, 10, 1), 
    'max_depth': np.arange(1, 11, 1)
}

forest_reg = RandomForestRegressor() 

grid_search = GridSearchCV(
    forest_reg, param_grid, cv = 5, 
    scoring = 'neg_mean_squared_error'
)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [6]:
# Get the best estimator's parameters
grid_search.best_params_ 

{'max_depth': 1, 'max_features': 2, 'n_estimators': 50}

In [7]:
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(data, prediction): 
  """
  Calculates root mean squared error of a prediction set

  :param data: Actual dataset
  :type data: numpy.ndArray

  :param prediction: Predicted values
  :type prediction: numpy.ndArray 

  :returs: Root mean squared errors for each instance
  :rtype: numpy.ndArray
  """
  return np.sqrt(mean_squared_error(data, prediction))

In [8]:
# Get the accuracy of the best model 

best_estimator = grid_search.best_estimator_

random_forest_predictions = best_estimator.predict(X_test)
print(root_mean_squared_error(y_test, random_forest_predictions))

0.8127308806870055


In [9]:
import joblib

# Export the best model 

best_model = grid_search.best_estimator_

filename = "por_time_pred.pkl"

joblib.dump(best_model, filename)

['por_time_pred.pkl']